<a href="https://colab.research.google.com/github/dsdayat/HW_Supervised_II_Random-Forest/blob/main/HW_Supervised_II_Agus_Nur_Hidayat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised ML in Python: Homework

In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Instruksi

Load dan siapkan dataset dengan menjalankan cell-cell berikut.

In [75]:
df = pd.read_csv('supervised_ml_homework_data.csv')

In [76]:
df.head()

,initial_rate,length,width,height,speed,y
0,0.12,5.87,7.31,9.61,24.47,0
1,0.68,5.49,5.46,9.19,20.33,0
2,1.86,6.03,5.89,10.75,22.60,0
3,0.04,5.25,5.57,10.04,20.00,0
4,0.28,6.63,6.17,9.90,21.74,0


In [77]:
df_train, df_test = train_test_split(df, 
                                     test_size = 0.25, random_state = 1) 

x_train_data = df_train.loc[:, df_train.columns!='y']
y_train_data = df_train.loc[:, df_train.columns=='y']

x_test_data = df_test.loc[:, df_test.columns!='y']
y_test_data = df_test.loc[:, df_test.columns=='y']

In [78]:
df.dtypes

initial_rate    float64
length          float64
width           float64
height          float64
speed           float64
y                 int64
dtype: object

Kemudian, lakukan langkah-langkah di bawah ini:
1. Fit and evaluate Random Forest Classifier biasa
2. Fit and evaluate Random Forest Classifier with Undersampling
3. Fit and evaluate Random Forest Classifier with Oversampling
4. Fit and evaluate Random Forest Classifier with Oversampling and Undersampling. 
5. Carilah nilai ideal ****berdasarkan F1-Score**** koefisien oversampling dan undersampling agar nilai F1-Score menjadi terbaik. 


Keterangan: cobalah nilai-nilai berikut

`oversampling_coef = [0.2,0.4,0.5,0.6,0.8]`

`undersampling_coef = [0.2,0.4,0.5,0.6,0.8]`

### Soal

1. Model dengan konfigurasi apa yang memiliki nilai recall terbaik?
2. Model dengan konfigurasi apa yang memiliki nilai precision terbaik?
3. Model dengan konfigurasi apa yang memiliki nilai F1-Score terbaik?

Jawablah dengan lengkap, misalnya:

- 'Model Random Forest biasa', atau

- 'Model Random Forest dengan Undersampling saja', atau

- 'Model Random Forest dengan Oversampling dan Undersampling dengan koefisien oversampling = 0.2 dan koefisien undersampling = 0.9'

### Pengerjaan

# Radom Forest Model Biasa

In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [80]:
def evaluate(model, x_test = x_test_data, y_test = y_test_data):
    y_pred = model.predict(x_test)    
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('------')
    print('Confusion Matrix:')
    print()
    print(classification_report(y_test, y_pred))
    print('------')
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print('Assuming 1 as positive, we get:')
    print('')
    print('True Positive:', tp)
    print('True Negative:', tn)
    print('False Positive:', fp)
    print('False Negative:', fn)
    
    print('')
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-Score:', 2 * (precision * recall)/(precision + recall))
    

In [81]:
### Base Model
base_model = RandomForestClassifier(random_state = 1, n_jobs = -1)
base_model.fit(x_train_data, y_train_data)
evaluate(base_model)

Accuracy: 0.952
------
Confusion Matrix:

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1130
           1       0.81      0.66      0.72       120

    accuracy                           0.95      1250
   macro avg       0.89      0.82      0.85      1250
weighted avg       0.95      0.95      0.95      1250

------
Assuming 1 as positive, we get:

True Positive: 79
True Negative: 1111
False Positive: 19
False Negative: 41

Precision: 0.8061224489795918
Recall: 0.6583333333333333
F1-Score: 0.7247706422018348


*Accuracy is quite high (0.952), Precission is quite good, recall is enough, and the F1-Score is moderate okkay*. 

remark :
- Mau mengurangi false negative >> pilih yg recall nya tinggi
- Mau mengurangi false positive >> pilih yg precisionnya tinggi
- Kalau mau balanced >> pilih yg F1-Score nya tinggi

- False Negetive stiil higher than false positive

# Random Forest Model Undersampling
- digunakan ketika model overfitting

In [82]:
# import random undersampling library 
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# summarize class distribution
print(Counter(y_train_data['y']))

# summarize class distribution
print(Counter(y_train_under['y']))

Counter({0: 3371, 1: 379})
Counter({0: 379, 1: 379})


In [83]:
### Undersampled Model 
# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy='majority',)
# fit and apply the transform
x_train_under, y_train_under = undersample.fit_resample(x_train_data, 
                                                        y_train_data) #silakan dilanjutkan

under_model = RandomForestClassifier(random_state = 1)

In [84]:
#Fit Model
under_model.fit(x_train_under, y_train_under)
#evaluation Model
evaluate(under_model, x_test_data, y_test_data)

Accuracy: 0.892
------
Confusion Matrix:

              precision    recall  f1-score   support

           0       1.00      0.88      0.94      1130
           1       0.47      0.97      0.63       120

    accuracy                           0.89      1250
   macro avg       0.73      0.93      0.78      1250
weighted avg       0.95      0.89      0.91      1250

------
Assuming 1 as positive, we get:

True Positive: 116
True Negative: 999
False Positive: 131
False Negative: 4

Precision: 0.46963562753036436
Recall: 0.9666666666666667
F1-Score: 0.6321525885558582


*The Accuracy is Good (0.8912), The Recall is quite high But The Precission is poor. So the F1 Score is in the lower of enough level*

---



- False Positive Extremely Increased, False Negative decreased

# Random Forest model Oversampling
- digunakan ketika model underfitting

In [85]:
# import random undersampling library 
from collections import Counter
from imblearn.over_sampling import SMOTE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# summarize class distribution
print(Counter(y_train_data['y']))

# summarize class distribution
print(Counter(y_train_SMOTE['y']))

Counter({0: 3371, 1: 379})
Counter({0: 3371, 1: 3371})


In [86]:
### Oversampled Model # define undersample strategy
SMOTE = SMOTE(random_state = 1)

# fit and apply the transform
x_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(x_train_data, y_train_data)
over_model = RandomForestClassifier(n_jobs = -1)
over_model.fit(x_train_SMOTE, y_train_SMOTE)

#Evaluate Model
evaluate(over_model, x_test_data, y_test_data)

Accuracy: 0.9328
------
Confusion Matrix:

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      1130
           1       0.61      0.82      0.70       120

    accuracy                           0.93      1250
   macro avg       0.80      0.88      0.83      1250
weighted avg       0.95      0.93      0.94      1250

------
Assuming 1 as positive, we get:

True Positive: 99
True Negative: 1067
False Positive: 63
False Negative: 21

Precision: 0.6111111111111112
Recall: 0.825
F1-Score: 0.7021276595744681


*The Accuracy is quite high (0.9352), The Recall is Good (0.83) But The Precission is in lower of enough level (0.62). and the F1 Score is enough*
- False Postive Increased, False negative Decreased


# Random Forest Classifier Model Combining  (Oversampling and Undersampling)

In [87]:
### Combined Model
### silakan dilanjutkan
### ingat, untuk model, untuk undersampler, dan oversampler...
### ...random_state = 1. 

In [88]:
# import needed libraries 
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# define pipeline
combined_model = RandomForestClassifier(random_state = 1, n_jobs = -1)
over = SMOTE(sampling_strategy=0.2)
under = RandomUnderSampler(sampling_strategy=0.2)
steps = [('o', over), ('u', under), ('model', combined_model)]
pipeline = Pipeline(steps=steps)

# evaluate pipeline
pipeline.fit(x_train_data, y_train_data)
evaluate(pipeline, x_test_data, y_test_data)

Accuracy: 0.9536
------
Confusion Matrix:

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1130
           1       0.77      0.73      0.75       120

    accuracy                           0.95      1250
   macro avg       0.87      0.86      0.86      1250
weighted avg       0.95      0.95      0.95      1250

------
Assuming 1 as positive, we get:

True Positive: 88
True Negative: 1104
False Positive: 26
False Negative: 32

Precision: 0.7719298245614035
Recall: 0.7333333333333333
F1-Score: 0.7521367521367521


*The Accuracy is quite high (0.9448), The Recall is Good (0.81) But The Precission is in lower of enough level (0.62). and the F1 Score is enough*

- Random Forest Combined model Undersampling and Undersampling has **the highest F1 Score (0.75)** than other  model
- False Postive Increased, False negative Decreased

### Jawaban

1. Model dengan konfigurasi apa yang memiliki nilai recall terbaik?

Jawab: *Random Forest Model Undersampling*


2. Model dengan konfigurasi apa yang memiliki nilai precision terbaik?

Jawab: *Random Forest Model biasa*


3. Model dengan konfigurasi apa yang memiliki nilai F1-Score terbaik?

Jawab: *Random Forest Classifier Model Combining  (Oversampling and Undersampling)* dengan over_coef = 0.2 dan under_coef = 0.2
(berdasarkan data dibawah)

In [89]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore')

oversampling_coef = [0.2,0.4,0.5,0.6,0.8]

undersampling_coef = [0.2,0.4,0.5,0.6,0.8]

result = {
    'over_coef':[],
    'under_coef':[],
    'acc':[],
    'prec':[],
    'recall':[],
    'f1':[]
}

for over_score in tqdm(oversampling_coef):
    for under_score in undersampling_coef:
        try:
            combined_model = RandomForestClassifier(random_state = 1, n_jobs = -1)
            over = SMOTE(sampling_strategy = over_score)
            under = RandomUnderSampler(sampling_strategy = under_score)
            steps = [('o', over), ('u', under), ('model', combined_model)]
            pipeline = Pipeline(steps = steps)

            pipeline.fit(x_train_data, y_train_data)
            y_pred = pipeline.predict(x_test_data)

            result['over_coef'].append(over_score)
            result['under_coef'].append(under_score)
            result['acc'].append(accuracy_score(y_test_data, y_pred))
            result['prec'].append(precision_score(y_test_data, y_pred))
            result['recall'].append(recall_score(y_test_data, y_pred))
            result['f1'].append(f1_score(y_test_data, y_pred))
        except:
            pass

100%|██████████| 5/5 [00:10<00:00,  2.09s/it]


In [90]:
df_result = pd.DataFrame(result)
df_result

,over_coef,under_coef,acc,prec,recall,f1
0,0.2,0.2,0.9560,0.768595,0.775000,0.771784
1,0.2,0.4,0.9408,0.647436,0.841667,0.731884
2,0.2,0.5,0.9376,0.625000,0.875000,0.729167
3,0.2,0.6,0.9304,0.592179,0.883333,0.709030
4,0.2,0.8,0.9224,0.557789,0.925000,0.695925
5,0.4,0.4,0.9472,0.701493,0.783333,0.740157
6,0.4,0.5,0.9408,0.649351,0.833333,0.729927
7,0.4,0.6,0.9416,0.655629,0.825000,0.730627
8,0.4,0.8,0.9344,0.610465,0.875000,0.719178
9,0.5,0.5,0.9432,0.676259,0.783333,0.725869


In [91]:
#cari koefisien over dan under yang terbaik terhadap F1 score
df_result.sort_values(['recall'], ascending=False)


,over_coef,under_coef,acc,prec,recall,f1
4,0.2,0.8,0.9224,0.557789,0.925000,0.695925
3,0.2,0.6,0.9304,0.592179,0.883333,0.709030
2,0.2,0.5,0.9376,0.625000,0.875000,0.729167
8,0.4,0.8,0.9344,0.610465,0.875000,0.719178
11,0.5,0.8,0.9344,0.614458,0.850000,0.713287
1,0.2,0.4,0.9408,0.647436,0.841667,0.731884
13,0.6,0.8,0.9312,0.601190,0.841667,0.701389
6,0.4,0.5,0.9408,0.649351,0.833333,0.729927
10,0.5,0.6,0.9400,0.645161,0.833333,0.727273
7,0.4,0.6,0.9416,0.655629,0.825000,0.730627


The model result : 

- The highest`precision`,  over coef = 0.2, under coef = 0.2
- The highest `recall`, over coef = 0.2, under coef = 0.6
- The highest `f1`, over coef = 0.2, under coef = 0.2